In [ ]:
"""
Method of curvature and tilt correction of calibration lines in high-resolution spectra
Author: Tanya Das
"""

In [ ]:
%matplotlib qt
import numpy as np

from astropy.io import fits
from astropy.convolution import convolve, Gaussian1DKernel

from scipy.signal import find_peaks
from scipy import signal,optimize,interpolate

from skimage.transform import warp, rotate
from skimage import filters
from skimage.color import label2rgb
from skimage import measure
from skimage.measure import label, regionprops, regionprops_table

from tqdm import tqdm 

import matplotlib.pyplot as plt
from matplotlib import style
import matplotlib
matplotlib.rcParams.update({'font.size': 13})
style.use('ggplot')

In [ ]:
"""Reads and display the fits file"""

fle = "hesp_ThAr_postproc.fit"
hdu= fits.open(fle)
sc = hdu[0].data
xc, yc = sc.shape[1], sc.shape[0]
print(xc,yc)

plt.figure(1, figsize=(10,8))
plt.imshow(sc, cmap=plt.cm.gray, vmin=0, vmax=100)    #change vmax value if the plot is not visible

In [ ]:
"""Reads the aperture trace polynomial coefficients from file"""

f3 = open("hesp_ThAr_trace.txt","r")
content=f3.readlines()
pcof = np.zeros((len(content),4))
flag = 0

for l in content:
    sl=l.split(" ")  
    
    pcof[flag][0]=float(sl[2])
    pcof[flag][1]=float(sl[3])
    pcof[flag][2]=float(sl[5])
    pcof[flag][3]=float(sl[7])  
    
    flag = flag+1

print(pcof.shape)
f3.close()   

In [ ]:
def ap_extract(coff, lt1):

    """
    Extracts and returns selected aperture from the spectrum by creating a mask using the trace polynomial coefficients.
    Binsize and binres are global variables.

    Parameters:
        - coff (ndarray)    : Aperture trace coefficients for the aperture to be extracted
        - lt1 (int)         : Length value along y axis
    
    Returns:
        - apert (ndarray)   : Extracted order from the spectrum
    """  
    fin_data = []  
    ft = []
    ftx = []

    for x0 in range(0,xc,1):
          
        y0=coff[0]*x0**3+coff[1]*x0**2+coff[2]*x0 + coff[3]
        ft = np.append(ft, y0)
        ftx = np.append(ftx, x0)
     
    for i in range(0,xc,1):
        data_temp = []
        dum = 0.0
        strt = ft[i]

        for pix in np.arange(strt-(binsize/2),(strt+(binsize/2))+binres,binres):
            dum = f(pix, i)   
            data_temp = np.append(data_temp,dum)

        if i == 0:

            fin_data = np.hstack((fin_data,data_temp[0:lt1]))


        else:
            fin_data = np.vstack((fin_data,data_temp[0:lt1]))

    apert = np.transpose(fin_data)
    
    return apert

In [ ]:
def background_stdmed(apert):
    """
    Calculates the standard deviation and median of the background in the extracted order.
    
    Parameters:
        -apert (ndarray)  : Extracted order from the spectrum
        
    Returns:
        -bck_std (float)  : standard deviation of the background in the spectrum
        -bck_med (float)  : median of the background in the spectrum        
    """
    bg_ar = []
    for i in range (0,yc2,1):
        for j in range(0,xc2,1):

            if apert[i][j] < 50:     #value decided based on the type of spectra, manually input by checking in spectrum
                bg_ar = np.append(bg_ar,apert[i][j])
            else:
                continue

    bck_std = np.std(bg_ar)
    bck_med = np.median(bg_ar)

    return bck_std, bck_med

In [ ]:
def boundaryArray(apert, stddev, bck_std, plm):
    """
    Returns an array with boundary values for individual FP/Th-Ar lines.
    It detects the line position after gaussian smoothening a y-cut through the middle of the order and
    finding the peaks in smoothened data. The peak positions are used as mid point to determine the boundary.
    
    Parameters:
        -apert (ndarray)  : Extracted order from the spectrum
        -stddev (float)   : Standard deviation of the 1D gaussian smoothening kernel. It's chosen based on the spectrograph
        -bck_std (float)  : standard deviation of the background in the spectrum
        -plm (int)        : Linewidth measured from data / 2
    
    Returns:
        -pks (ndarray)      : Boundary array for spectrum lines
        -bdry (ndarray)   : Boundary array for continuum    
    """
    x = np.arange(0,xc2)
    x_pos = []
    y_max = []
    pks = []
    bdry = []
    
    xct, yct = int(xc2/2), int(yc2/2)
    lin_ct = apert[yct,:]
    med_lin = np.median(lin_ct)    
    
    gauss_kernel = Gaussian1DKernel(stddev)   
    fp_sm = convolve(lin_ct, gauss_kernel) 
    
    ind,pht = find_peaks(fp_sm,height=2*bck_std,distance=6)  #distance will vary depending on spectrum
    x_pos = x[ind]
    y_max=lin_ct[ind] # Corresponding Intensity
    
    if ind[0]<plm:  #value given to avoid edge values.
        ind = np.delete(ind,0)
    
    l = len(x_pos)
    
    for i in range(0,l):
        x1 = ind[i] - plm    
        x2 = ind[i] + plm
        pks = np.hstack((pks,x1,x2))
    

    bdry = np.hstack((bdry,x[0]))
    bdry = np.hstack((bdry,pks))
    bdry = np.hstack((bdry,x[-1]))  
    
    """  
    #works only for FP
    adj_df = np.diff(x_pos)
    med_len = np.median(adj_df)
    if (med_len % 2) == 0:
        N = med_len / 2
    else:
        N = (med_len+1) / 2

    plm=N
"""    
    
    return pks, bdry

In [ ]:
def fit_slopeN(xs,ys):
    """
    Calculates the slope of individual slanted line in spectrum
    
    Parameters:
        -xs (ndarray) : x-array for calculating slope
        -ys (ndarray) : y-array for calculating slope
        
    Returns:
        -m (float)    : calculated slope value
        -b (float)    : calculated intercept value     
    """    
    N = len(xs)
    meanx = np.mean(xs)
    meany = np.mean(ys)
    num = 0
    den = 0
    for i in range(N):
        num += (xs[i] - meanx) * (ys[i] - meany)
        den += ((xs[i] - meanx) ** 2)
    
    m = num / den
    b = meany - (m * meanx)
    
    if (np.isnan(m)):
        return 0,b
    else:
        return m, b

In [ ]:
def shift_norm(xy,center,m):
    """
    Calculates the new coordinate values of the isolated line, by shifting the pixels to correct the slant in the line
    
    Parameters:
        -xy (ndarray)      : Original coordinates in the isolated line
        -center (ndarray)  : Center value in the original isolated line
        -m (float)         : Slope value corresponding to the slant in line
        
    Returns:
        - xy(ndarray)      : new coordinates corresponding to the slant corrected line     
    """
    xt,yt = np.transpose(xy)
    x0, y0 = center
    
    shf = (yt - y0)*m
    newx = xt + shf
    
    xy[...,0] = newx
    xy[...,1] = yt
    
    return xy

In [ ]:
def slant_correct(rect):
    """
    Performs slant correction in the selected line in the spectrum by using the warp function from
    Scikit-Image processing package
    
    Parameters:
        -rect (ndarray)     : Individual slanted line in the spectrum
        
    Returns:
        - warped1 (ndarray) : Slant corrected line    
    """  
    indx = []
    ys = []
    ysel = rect
    y_ext = []
    indfx = []
    indy = []  

    threshold_value = filters.threshold_otsu(rect)
    labeled_foreground = (rect > threshold_value).astype(int)
    properties = regionprops(labeled_foreground,rect)
    slope_angle = properties[0].orientation     #use for FP
    weighted_center_of_mass = properties[0].weighted_centroid

    x0 = weighted_center_of_mass[1]  #use np.median(xs) if using slope method weighted_center_of_mass[1] for regionprop method       
    y0 = rect.shape[0]/2

    center = (x0,y0)
    
    warp_args = {'center': center,
                     'm': slope_angle,
                    }      

    warped1 = warp(rect, shift_norm, map_args=warp_args, order=3, mode='wrap')
    
    """    
#manually calculates the slant of line
    for loop1 in range(0,rect.shape[0],1):

        pk1 = ysel[loop1,:]
        stdev = np.std(pk1)
        
        ind1,ht1 = find_peaks(pk1,height=2*stdev,distance=5)
    
        if(ind1.size>0):
            ind1 = max(ind1)
            if (pk1[ind1] > 50):   #value decided based on the type of spectra, manually input by checking in spectrum
                indfx = np.append(indfx,ind1)
                indy = np.append(indy,loop1)

        else:
            continue

    xs = indfx
    ys = indy
    xs = np.append(xs,weighted_center_of_mass[1])
    ys = np.append(ys,weighted_center_of_mass[0])
     
    mN,bN = fit_slopeN(ys,xs)
"""

    return warped1

In [ ]:
def corrected_ap(apert, pks, bdry):
    """
    Selects the region of interest by isolating individual lines in the order using the boundary array and saves
    the slant corrected lines in the corrected aperture
    
    Parameters:
        -apert (ndarray)      : Extracted order from the spectra
        -pks (ndarray)        : Boundary array for spectra lines
        -bdry (ndarray)       : Boundary array for continuum
        
    Returns:
        -final_img (ndarray)  : Individual line slant corrected order   
    """
    k = 0
    y2 = yc2
    x_max = xc2

    final_img = np.zeros((yc2,x_max))

    while k < len(pks): 

#         print(k)
        x1 = int(pks[k])
        x2 = int(pks[k+1])
        r_sel = apert[:,x1:x2]

        X1 = int(bdry[k])
        X2 = int(bdry[k+1])
        cnt_bdry = apert[:,X1:X2]

        warp1 = slant_correct(r_sel)

        final_img[:,X1:X2] = cnt_bdry
        final_img[:,x1:x2] = warp1

        k = k+2
        
    final_img[:,X2:int(bdry[-1])]=apert[:,X2:int(bdry[-1])]

    return final_img

In [ ]:
"""The binsize and binres can be decided according to the spectra in order to avoid overlapping of the extraction mask"""
binsize = 17
binres = 0.5
plm = 4     #the +/- values can be chosen based on line width in data and halved

selcof = pcof[10:18,:]  #to avoid corrupted orders, this step can be omitted in case full spectra is required

xaxis=np.linspace(0,xc,xc)
yaxis=np.linspace(0,yc,yc)
f=interpolate.RectBivariateSpline(yaxis, xaxis, sc)

lt1 = int((binsize/binres)+1)
cflen = len(selcof)
y_ex = lt1*cflen

fin_list = np.empty((y_ex,xc))

for j in tqdm(range(cflen)):
    
    temp_apert = []
    coff = selcof[j]
    
    temp_apert = ap_extract(coff, lt1)
    l = len(temp_apert)
    
    xc2, yc2 = temp_apert.shape[1], temp_apert.shape[0] 

    back_std, back_med = background_stdmed(temp_apert)
    line_bdry, cont_bdry = boundaryArray(temp_apert, 1.0, back_std, plm)
    slant_cor = corrected_ap(temp_apert, line_bdry, cont_bdry)

    fin_list[(j*l):(j+1)*l, :] = slant_cor

In [ ]:
plt.figure(2, figsize=(10,8))
plt.imshow(fin_list, cmap = 'gray', interpolation = 'nearest', vmin=0, vmax=100)